# This notebook is for testing purposes , It is not the final deliverable.
## Contents:
0. [Message Encoding with message blocks](#0)
1. [Diffie-Hellman Key Exchange](#1)
2. [AlGamal](#2)
3. [Initilization](#3)
4. [Handling Messages](#4)
5. [Integration to chat.py](#5)

0. [Message Encoding with message blocks](#0)

### Import relevant libraries

In [10]:
import base64

from helpers.dh_gamal import deffie_hellman , secret_key_from_dh
from helpers.tools import generate_prime
from helpers.tools import from_file, generate_params_file

%reload_ext autoreload
%autoreload 2

### Message Encoding with message blocks

The main reason to use Base64 is to be trasmit data over a channel that doesn't allow binary data transfer. That said, now it should be clear that you can have a stream encoded in Base64 that rapresent a stream UTF8 encoded.

[What is the difference between utf-8 and base64](https://stackoverflow.com/questions/3866316/whats-the-difference-between-utf8-utf16-and-base64-in-terms-of-encoding#:~:text=But%20in%20languages%20where%20there's,to%20keep%20the%20concerns%20separate.&text=%2B1%20Base64%20is%20a%20way,ways%20to%20encode%20Unicode%20text.)

#### Trying different encodings and decodings

In [ ]:
message = "hello world hello baba hello mama hello teta hello geddo"

In [ ]:
print("Original Message: " , message)

print("#############################################################")
print("UTF-ENCDOING")
print("#############################################################")
print("Message with UTF-8 encoding: ",message.encode('utf-8'))
print("Message with UTF-16 encoding: ",message.encode('utf-16'))
print("Message with UTF-32 encoding: ",message.encode('utf-32'))


print("#############################################################")
print("INT.FROM_BYTES little endian encoding")
print("#############################################################")
# testing int.from_bytes
print(int.from_bytes(message.encode('utf-8'), 'little'))
print(int.from_bytes(message.encode('utf-16'), 'little'))
print(int.from_bytes(message.encode('utf-32'), 'little'))


print("#############################################################")
print("Base64 Encoding")
print("#############################################################")
# encode with base64
message_bytes = message.encode('utf-8')
print("Message with base64 encoding: ", base64.b64encode(message_bytes))
print("Message with a85 encoding: ", base64.a85encode(message_bytes))
print("Message with b85 encoding: ", base64.b85encode(message_bytes))
print("Message with b16 encoding: ", base64.b16encode(message_bytes))
print("Message with b32 encoding: ", base64.b32encode(message_bytes))
print("Message with b85 encoding: ", base64.b85encode(message_bytes))
print("Message with b32hex encoding: ", base64.b32hexencode(message_bytes))
print("Message with standard base64 encoding: ", base64.standard_b64encode(message_bytes))
print("Message with urlsafe base64 encoding: ", base64.urlsafe_b64encode(message_bytes))

print("#############################################################")
print("INT FROM_BYTES little endian encoding")
print("#############################################################")
# testing int.from_bytes
print(int.from_bytes(base64.b64encode(message_bytes), 'little'))
print(int.from_bytes(base64.a85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b16encode(message_bytes), 'little'))
print(int.from_bytes(base64.b32encode(message_bytes), 'little'))
print(int.from_bytes(base64.b85encode(message_bytes), 'little'))
print(int.from_bytes(base64.b32hexencode(message_bytes), 'little'))
print(int.from_bytes(base64.standard_b64encode(message_bytes), 'little'))
print(int.from_bytes(base64.urlsafe_b64encode(message_bytes), 'little'))

print(message.encode('utf-8').decode('utf-8'))
print(base64.b85decode(base64.b85encode(message.encode('utf-8'))).decode('utf-8'))

In [14]:
generate_params_file("agree.txt")

TypeError: generate_prime() takes 2 positional arguments but 3 were given

### 1. [Diffie-Hellman Key Exchange](#1)

In [13]:
q_dh , a_dh , _ , _ = from_file('agree.txt')
X_a = generate_prime(2,q_dh,1)
print("Alice Private Key: ", X_a)
X_b = generate_prime(2, q_dh,1)
print("Bob Private Key: ", X_b)
Y_a = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_a)
print("Alice Public Key: ", Y_a)
Y_b = deffie_hellman(prime=q_dh, generator=a_dh, private_key=X_b)
print("Bob Public Key: ", Y_b)
Ka = secret_key_from_dh(prime=q_dh, public_key=Y_b, private_key=X_a)
Kb = secret_key_from_dh(prime=q_dh, public_key=Y_a, private_key=X_b)
print("Alice Secret Key and Bob Secret Key: ", Ka, Kb)



Alice Private Key:  5
Bob Private Key:  211
Alice Public Key:  91
Bob Public Key:  20
Alice Secret Key and Bob Secret Key:  202 202


### 2. [AlGamal](#2)

### 3. [Initilization](#3)

### 4. [Handling Messages](#4)

### 5. [Integration to chat.py](#5)